In [2]:
from fastHan import FastHan
model = FastHan(model_type='base')

  0%|          | 197k/144M [00:00<01:21, 1.77MB/s]

http://212.129.155.247/fasthan/fasthan_base.zip not found in cache, downloading to /tmp/tmp0o9tq_p_


100%|██████████| 144M/144M [00:19<00:00, 7.54MB/s] 


Finish download from http://212.129.155.247/fasthan/fasthan_base.zip
Copy file to /home/manman/.fastNLP/fasthan/fasthan_base
loading vocabulary file /home/manman/.fastNLP/fasthan/fasthan_base/vocab.txt
Load pre-trained BERT parameters from file /home/manman/.fastNLP/fasthan/fasthan_base/model.bin.


In [3]:
import jieba

### target 参数可在 
## {'Parsing'=依存分析、'CWS'=分词、'POS'=词性标注、'NER'=命名实体识} 
#### 四个选项中取值，模型将分别进行依存分析、分词、词性标注、命名实体识别任务,
#### 模型默认进行 CWS 任务。其中词性标注任务包含了分词的信息，
#### 而依存分析任务又包含了词性标注任务的信息。
#### 命名实体识别任务相较其他任务独立。

In [4]:
sentence = ["我是张慢慢","我是张小伴的爸爸"]
answer = model(sentence,target='Parsing')
answer

[[['我', 2, 'top', 'PN'], ['是', 0, 'root', 'VC'], ['张', 2, 'attr', 'M'], ['慢慢', 2, 'advmod', 'AD']],
 [['我', 2, 'top', 'PN'], ['是', 0, 'root', 'VC'], ['张小伴', 5, 'assmod', 'NR'], ['的', 3, 'assm', 'DEG'], ['爸爸', 2, 'attr', 'NN']]]

#### 模型的输出是在 fastHan 模块中定义的 sentence 与 token 类。
#### 模型将输出一个由 sentence 组成的列表，而每个 sentence 又由 token 组成。每个 token 本身代表一个被分好的词，有 pos、head、head_label、ner 四项属性，代表了该词的词性、依存关系、命名实体识别信息。

#### 如果分别运行 CWS、POS、Parsing 任务，模型输出的分词结果等可能存在冲突。如果想获得不冲突的各类信息，可以直接运行包含全部所需信息的那项任务。

#### 模型的 POS、Parsing 任务均使用 CTB 标签集。NER 使用 msra 标签集。

#### 模型在 13 个语料库中进行训练，其中包含 10 个分词语料库。不同语料库的分词粒度均不同，如本模型默认的 CTB 语料库分词粒度较细。如果想切换不同的粒度，可以使用模型的 set_cws_style 函数，例子如下

In [5]:
sentence2 = '一个苹果手机'
model(sentence2,target='CWS')

[['一', '个', '苹果', '手机']]

In [6]:
list(jieba.cut_for_search(sentence2))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.511 seconds.
Prefix dict has been built successfully.


['一个', '苹果', '手机']

In [7]:
model.set_cws_style('cnc')
model(sentence2,'CWS')

[['一个', '苹果', '手机']]

In [8]:
model.set_cws_style('cnc')
model._get_list(sentence2,tags='CWS')

['苹']

# GPU模式

In [9]:
model.set_device('cuda:0')

# CPU 模式

In [10]:
model.set_device('cpu')

## 繁體字分詞

In [11]:
sentence3 = '今天天氣還不錯，要不我們出去走走怎麼樣？'
model.set_cws_style('as')
model(sentence3,target='CWS')

[['今天', '天氣', '還', '不錯', '，', '要不', '我們', '出去', '走走', '怎麼樣', '？']]

In [12]:
model.set_cws_style('cityu')
model(sentence3,target='CWS')

[['今天', '天氣', '還', '不錯', '，', '要不', '我們', '出去', '走走', '怎麼樣', '？']]

## 模型在以下数据集进行测试和训练：

## CWS：AS、CITYU、CNC、CTB、MSR、PKU、SXU、UDC、WTB、ZX

## NER：MSRA、OntoNotes

## POS & Parsing：CTB9

### 注：模型在训练 NER OntoNotes 时将其标签集转换为与 MSRA 一致。

## 最终模型在各项任务中取得的 F 值如下：
<img src="/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/im/7FE7B1F4-0B20-4796-91C8-37818A69DFE6.png
">

## 表格中单位为百分数。CWS 的成绩是 10 项任务的平均成绩。Parsing 中的两个成绩分别代表 F_{ldep} 和 F_{udep}。SOTA 模型的数据来自笔者对网上资料及论文的查阅，如有缺漏请指正，不胜感激。这五项 SOTA 表现分别来自如下五篇论文：